In [1]:
import requests
import pandas as pd

In [2]:
response = requests.get('https://api.data.codeup.com/api/v1/stores')

In [4]:
type(response)

requests.models.Response

In [5]:
data = response.json()

In [6]:
data

{'payload': {'max_page': 1,
  'next_page': None,
  'page': 1,
  'previous_page': None,
  'stores': [{'store_address': '12125 Alamo Ranch Pkwy',
    'store_city': 'San Antonio',
    'store_id': 1,
    'store_state': 'TX',
    'store_zipcode': '78253'},
   {'store_address': '9255 FM 471 West',
    'store_city': 'San Antonio',
    'store_id': 2,
    'store_state': 'TX',
    'store_zipcode': '78251'},
   {'store_address': '2118 Fredericksburg Rdj',
    'store_city': 'San Antonio',
    'store_id': 3,
    'store_state': 'TX',
    'store_zipcode': '78201'},
   {'store_address': '516 S Flores St',
    'store_city': 'San Antonio',
    'store_id': 4,
    'store_state': 'TX',
    'store_zipcode': '78204'},
   {'store_address': '1520 Austin Hwy',
    'store_city': 'San Antonio',
    'store_id': 5,
    'store_state': 'TX',
    'store_zipcode': '78218'},
   {'store_address': '1015 S WW White Rd',
    'store_city': 'San Antonio',
    'store_id': 6,
    'store_state': 'TX',
    'store_zipcode': '78220

In [7]:
stores = pd.DataFrame(data['payload']['stores'])

In [8]:
stores

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


In [9]:
response = requests.get('https://api.data.codeup.com/api/v1/items')
data = response.json()
data

{'payload': {'items': [{'item_brand': 'Riceland',
    'item_id': 1,
    'item_name': 'Riceland American Jazmine Rice',
    'item_price': 0.84,
    'item_upc12': '35200264013',
    'item_upc14': '35200264013'},
   {'item_brand': 'Caress',
    'item_id': 2,
    'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
    'item_price': 6.44,
    'item_upc12': '11111065925',
    'item_upc14': '11111065925'},
   {'item_brand': 'Earths Best',
    'item_id': 3,
    'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
    'item_price': 2.43,
    'item_upc12': '23923330139',
    'item_upc14': '23923330139'},
   {'item_brand': 'Boars Head',
    'item_id': 4,
    'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
    'item_price': 3.14,
    'item_upc12': '208528800007',
    'item_upc14': '208528800007'},
   {'item_brand': 'Back To Nature',
    'item_id': 5,
    'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
    'item_price':

The JSON object contains details about the current, next, and max page. Very convenient. 

Two possible approaches to this API:
- loop until we don't have a next page
- make 1 request and figure out the max page, then loop over all the pages

In [11]:
# Quick side lesson on how python inteprets the presence/absence of a value as a boolean
if 'santa':
    print("Ho Ho Ho!")

Ho Ho Ho!


In [12]:
if None:
    print("This won't show up")

Within the scope of control flow keywords (`if`, `while`), python interprets the presence of an object as `True`

This doesn't invalidate everything you already know about using comparison operators.

In [13]:
'santa' == True

False

In [15]:
# Approach style 1 - Loop until we can't anymore
endpoint = "items"
host = "https://python.zgulde.net/"
api = "api/v1/"

url = host + api + endpoint

response = requests.get(url)

In [16]:
response.ok

True

In [17]:
if response.ok:
    payload = response.json()["payload"]

    # endpoint should be "items", "sales", or "stores"
    contents = payload[endpoint]

    # Make a dataframe of the contents
    df = pd.DataFrame(contents)

    next_page = payload["next_page"]

    # next_page is None when we're on the last page.
    # This loop only runs if there is a next_page defined.
    while next_page:
        # Append the next_page url piece
        url = host + next_page
        response = requests.get(url)

        payload = response.json()["payload"]

        next_page = payload["next_page"]    
        contents = payload[endpoint]

        df = pd.concat([df, pd.DataFrame(contents)])

        df = df.reset_index(drop=True)

df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [18]:
# Refactor the code above (i.e. another way to do the same exact thing):
domain = 'https://api.data.codeup.com'
endpoint = '/api/v1/items'
items = []

while True:
    url = domain + endpoint
    response = requests.get(url)
    data = response.json()
    print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end='')
    items.extend(data['payload']['items'])
    endpoint = data['payload']['next_page']
    if endpoint is None:
        break

Getting page 3 of 3: https://api.data.codeup.com/api/v1/items?page=3

In [19]:
# Approach style 2, find the range of pages and iterate through that range
base_url = 'https://api.data.codeup.com/api/v1/sales?page='
sales = []

# make the first request and define variables for your range
# Also save results from first request
url = base_url + '1'
response = requests.get(url)
data = response.json()
max_page = data['payload']['max_page']
sales.extend(data['payload']['sales']) # .extend is similar to append but can bring in more than one item

# We already made the request to the first page, so we'll start at 2.
# We add 1 to max_page because the range() function is exclusive of the endpoint
page_range = range(2, max_page + 1)

for page in page_range:
    url = base_url + str(page)
    print(f'\rFetching page {page}/{max_page} {url}', end='') # By default, the print function ends with a newline. We can use end to change this behavior
    response = requests.get(url)
    data = response.json()
    sales.extend(data['payload']['sales'])

Fetching page 183/183 https://api.data.codeup.com/api/v1/sales?page=183

In [20]:
sales = pd.DataFrame(sales)

## "Dry-out" the process into a function
- DRY is a principle "Don't Repeat Yourself"
- We often build functions once we know that the code we want is reusable and not bespoke
- I'll build a function that takes in the string "sales", "items", or "stores"

In [21]:
def get_all(endpoint):
    """ Read all records on all pages """
    
    if endpoint not in ["sales", "items", "stores"]:
        return "Not available from this API. Check the documentation"
    
    host = "https://python.zgulde.net/"
    api = "api/v1/"

    url = host + api + endpoint

    response = requests.get(url)

    if response.ok:
        payload = response.json()["payload"]

        # endpoint should be "items", "sales", or "stores"
        contents = payload[endpoint]

        # Make a dataframe of the contents
        df = pd.DataFrame(contents)

        next_page = payload["next_page"]

        while next_page:
            # Append the next_page url piece
            url = host + next_page
            response = requests.get(url)

            payload = response.json()["payload"]

            next_page = payload["next_page"] 
            
            print(f'\rGetting page {payload["page"]} of {payload["max_page"]}: {url}', end='')      
            
            contents = payload[endpoint]

            df = pd.concat([df, pd.DataFrame(contents)])

            df = df.reset_index(drop=True)
            
    return df

### Exercise 1
Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [22]:
items = get_all("items")
items.head()

Getting page 3 of 3: https://python.zgulde.net//api/v1/items?page=3

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


### Exercise 2
Do the same thing, but for stores

In [23]:
stores = get_all('stores')
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


### Exercise 3
Get all the sales information from the same API

In [24]:
sales = get_all("sales")
sales.head()

Getting page 183 of 183: https://python.zgulde.net//api/v1/sales?page=183

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [25]:
sales.shape

(913000, 5)

### Exercise 4
Write all the 3 dataframes to disk as .csv files

In [26]:
# Write each dataframe to disk
items.to_csv("items.csv")
stores.to_csv("stores.csv")
sales.to_csv("sales.csv")

In [ ]:
# Run this cell to tell your gitignore to ignore all CSV files 
# The ! in a jupyter notebook is a way to issue commands to the command line
!echo "*.csv" >> .gitignore

### Exercise 5
Combine the data from your three separate dataframes into one large dataframe.

Look for nulls to see if I need a left-join or if inner will work

In [27]:
sales.isna().sum()

item           0
sale_amount    0
sale_date      0
sale_id        0
store          0
dtype: int64

In [28]:
items.isna().sum()

item_brand    0
item_id       0
item_name     0
item_price    0
item_upc12    0
item_upc14    0
dtype: int64

In [29]:
stores.isna().sum()

store_address    0
store_city       0
store_id         0
store_state      0
store_zipcode    0
dtype: int64

In [30]:
sales_and_stores = pd.merge(sales, 
                        stores,
                        how="inner",
                        left_on="store",
                        right_on="store_id")
sales_and_stores.head()

,item,sale_amount,sale_date,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [31]:
# Merge everything together
everything = pd.merge(sales_and_stores,
                      items,
                      how="inner",
                      left_on="item",
                      right_on="item_id")
everything.head()

,item,sale_amount,sale_date,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [32]:
everything.shape

(913000, 16)

### Exercise 6
Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017.

In [ ]:
electricity_consumption = pd.read_csv("https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv")
electricity_consumption.head()

In [ ]:
electricity_consumption.info()

In [ ]:
electricity_consumption.isna().sum()

In [ ]:
electricity_consumption.shape

In [ ]:
electricity_consumption